create table sales (id_venda int primary key, id_prod_caract int, nome_adm varchar, nome_produto varchar, id_funcionario int, nome_funcionario varchar, grupo int, cota int, projeto varchar, nome_cliente varchar, 
					data_venda date, data_venda_abv varchar, data_primeira_parcela date, data_ultima_parcela date, valor_venda numeric, taxa_comissao numeric, valor_comissao numeric, valor_liquido numeric, 
					company_id varchar);
					
create table projects (id_projeto int primary key, nome_adm varchar, nome_projeto varchar);

create table employees (id_funcionario int primary key, nome_funcionario varchar, cpf varchar, data_adesao date, salario numeric, pix varchar,
						projeto varchar, emp_validation bool, company_id varchar);
						
create table products (id_produto int primary key, nome_adm varchar, nome_produto varchar, company_id varchar);

create table prod_caract (id_prod_caract int primary key, nome_adm varchar, nome_produto varchar, projeto varchar, taxa_comissao numeric, qtd_parcelas int, taxa_parcelas varchar, company_id varchar);
					   
create table expenses (id_despesa int primary key, tipo_despesa varchar, nome_despesa varchar, qtd int, descricao varchar, valor_salario numeric, valor_comissao numeric, valor_despesa numeric, data_despesa date, 
					  data_despesa_abv varchar, company_id varchar);

create table administrators (id_adm int primary key, nome_adm varchar, company_id varchar);

In [4]:
lista = ['a','b','c','d']
idx = lista.index('e') if 'e' in lista else 0
idx

0

In [8]:
lista.sort()

In [1]:
import psycopg2
import pandas as pd
import numpy as np
from datetime import datetime as dt

In [2]:
conn = psycopg2.connect(dbname='postgres', user='postgres', password='1673', host='localhost', port='5432')
cur = conn.cursor()

In [21]:
query = f"select nome_produto from products;"
cur.execute(query)
resp = cur.fetchall()
list_prod = [x[0] for x in resp]

# for prod in list_prod:
#     query = f"select * from income where nome_produto = '{prod}';"
#     cur.execute(query)
#     resp = cur.fetchall()
#     if len(resp) == 0:
#         print(prod)

GSC ACIMA 3M
a


In [58]:
value = 2.009
res = f'{value:_.2f}'.replace('.',',').replace('_','.')
res

'2,01'

In [5]:
def convert_column_types(df, list_float_col, list_date_col):
    df_view = df.copy()
    # df_view[list_float_col] = df_view[list_float_col].astype(str)
    for col in list_float_col:
        df_view[col] = df_view[col].map(lambda x: f'{x:_.2f}'.replace('.',',').replace('_','.'))
    for col in list_date_col:
        df_view[col] = df_view[col].map(lambda x: dt.strftime(x, '%d/%m/%Y'))
    return df_view

In [16]:
df_view = convert_column_types(df_sales, ['valor_venda','taxa_comissao','valor_comissao','taxa_receita','valor_receita'], ['data_venda','data_primeira_parcela','data_ultima_parcela'])
dict_rename_sales = {'id_venda':'ID Venda','nome_adm':'Adm','nome_produto':'Produto','nome_funcionario':'Vendedor','grupo':'Grupo','cota':'Cota','projeto':'Projeto','nome_cliente':'Cliente','data_venda':'Data Venda',
                'valor_venda':'Valor Venda (R$)','taxa_comissao':'Taxa Comissão (%)','valor_comissao':'Valor Comissão','taxa_receita':'Taxa Receita','valor_receita':'Valor Receita'}
df_view_upd = df_view[dict_rename_sales.keys()]
df_view_upd = df_view_upd.rename(columns=dict_rename_sales)

In [17]:
df_view_upd

,ID Venda,Adm,Produto,Vendedor,Grupo,Cota,Projeto,Cliente,Data Venda,Valor Venda (R$),Taxa Comissão (%),Valor Comissão,Taxa Receita,Valor Receita
0,2,BR,BR - Serv,Andyara,43,122,Especial 1,a,27/06/2024,"30.000,00","1,70","510,00","4,00","1.200,00"
1,3,Remaza,RMZ Big 75,Andyara,44,1,Especial 3,b,29/06/2024,"10.000,00","1,50","150,00","4,50","450,00"
2,4,Santander,SAN,Andyara,0,0,Especial 2,b,02/06/2024,"10.000,00","1,50","150,00","4,00","400,00"


In [7]:
company_id = 'contato_4b42bcf5-6cb5-47d7-8a4e-6c14b30f7dac'
cur.execute(f"SELECT * FROM sales WHERE company_id = '{company_id}';")
sales_list = list(cur.fetchall())
table_sales_columns = [desc[0] for desc in cur.description]
df_sales = pd.DataFrame(sales_list, columns=table_sales_columns)
df_sales.drop(columns=['company_id'], inplace=True)

df_sales['data_venda'] = pd.to_datetime(df_sales['data_venda'], format='%Y-%m-%d')
# df_sales['data_venda'] = df_sales['data_venda'].map(lambda x: x.date())
df_sales['data_primeira_parcela'] = pd.to_datetime(df_sales['data_primeira_parcela'], format='%Y-%m-%d')
df_sales['data_primeira_parcela'] = df_sales['data_primeira_parcela'].map(lambda x: x.date())
df_sales['data_ultima_parcela'] = pd.to_datetime(df_sales['data_ultima_parcela'], format='%Y-%m-%d')
df_sales['data_ultima_parcela'] = df_sales['data_ultima_parcela'].map(lambda x: x.date())
# df_sales[['data_venda','data_primeira_parcela','data_ultima_parcela']] = df_sales[['data_venda','data_primeira_parcela','data_ultima_parcela']].astype('datetime64[ns]')
df_sales[['valor_venda','taxa_comissao','valor_comissao','taxa_receita','valor_receita']] = df_sales[['valor_venda','taxa_comissao','valor_comissao','taxa_receita','valor_receita']].astype(float)

In [8]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id_venda               3 non-null      int64         
 1   id_prod_caract         3 non-null      int64         
 2   nome_adm               3 non-null      object        
 3   nome_produto           3 non-null      object        
 4   id_funcionario         3 non-null      int64         
 5   nome_funcionario       3 non-null      object        
 6   grupo                  3 non-null      int64         
 7   cota                   3 non-null      int64         
 8   projeto                3 non-null      object        
 9   nome_cliente           3 non-null      object        
 10  data_venda             3 non-null      datetime64[ns]
 11  data_venda_abv         3 non-null      object        
 12  data_primeira_parcela  3 non-null      object        
 13  data_ulti

In [10]:
year_selected = 2024
month_selected = 6
df_sales.loc[(df_sales['data_venda'].dt.year == int(year_selected)) & (df_sales['data_venda'].dt.month == int(month_selected))]

,id_venda,id_prod_caract,nome_adm,nome_produto,id_funcionario,nome_funcionario,grupo,cota,projeto,nome_cliente,data_venda,data_venda_abv,data_primeira_parcela,data_ultima_parcela,valor_venda,taxa_comissao,valor_comissao,taxa_receita,valor_receita,id_income
1,2,24,Magalu,MGL - Auto/Cam B50 B75,3,Roberto Andrade,1,1,Especial 1,a,2024-06-03,2024/06,2024-07-10,2025-03-10,111111111.0,1.4,1555555.55,1.8,2000000.0,4


In [52]:
df_sales

,id_venda,id_prod_caract,nome_adm,nome_produto,id_funcionario,nome_funcionario,grupo,cota,projeto,nome_cliente,data_venda,data_venda_abv,data_primeira_parcela,data_ultima_parcela,valor_venda,taxa_comissao,valor_comissao,taxa_receita,valor_receita,id_income
0,2,10,BR,BR - Serv,1,Andyara,43,122,Especial 1,a,2024-06-27,2024/06,2024-07-10,2024-12-10,30000.0,1.7,510.0,4.0,1200.0,2
1,3,306,Remaza,RMZ Big 75,1,Andyara,44,1,Especial 3,b,2024-06-29,2024/06,2024-07-10,2025-07-10,10000.0,1.5,150.0,4.5,450.0,46
2,4,319,Santander,SAN,1,Andyara,0,0,Especial 2,b,2024-06-02,2024/06,2024-07-10,2024-12-10,10000.0,1.5,150.0,4.0,400.0,48


In [53]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id_venda               3 non-null      int64  
 1   id_prod_caract         3 non-null      int64  
 2   nome_adm               3 non-null      object 
 3   nome_produto           3 non-null      object 
 4   id_funcionario         3 non-null      int64  
 5   nome_funcionario       3 non-null      object 
 6   grupo                  3 non-null      int64  
 7   cota                   3 non-null      int64  
 8   projeto                3 non-null      object 
 9   nome_cliente           3 non-null      object 
 10  data_venda             3 non-null      object 
 11  data_venda_abv         3 non-null      object 
 12  data_primeira_parcela  3 non-null      object 
 13  data_ultima_parcela    3 non-null      object 
 14  valor_venda            3 non-null      float64
 15  taxa_comis

In [54]:
for col in df_sales.columns:
    print(type(df_sales[col][0]))

<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'str'>
<class 'str'>
<class 'numpy.int64'>
<class 'str'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'str'>
<class 'str'>
<class 'datetime.date'>
<class 'str'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.int64'>


In [42]:
df_sales

,id_venda,id_prod_caract,nome_adm,nome_produto,id_funcionario,nome_funcionario,grupo,cota,projeto,nome_cliente,data_venda,data_venda_abv,data_primeira_parcela,data_ultima_parcela,valor_venda,taxa_comissao,valor_comissao,taxa_receita,valor_receita,id_income
0,2,10,BR,BR - Serv,1,Andyara,43,122,Especial 1,a,2024-06-27,2024/06,2024-07-10,2024-12-10,30000.0,1.7,510.0,4.0,1200.0,2
1,3,306,Remaza,RMZ Big 75,1,Andyara,44,1,Especial 3,b,2024-06-29,2024/06,2024-07-10,2025-07-10,10000.0,1.5,150.0,4.5,450.0,46
2,4,319,Santander,SAN,1,Andyara,0,0,Especial 2,b,2024-06-02,2024/06,2024-07-10,2024-12-10,10000.0,1.5,150.0,4.0,400.0,48


In [27]:
df_sales[['valor_venda','taxa_comissao','valor_comissao','taxa_receita','valor_receita']] = df_sales[['valor_venda','taxa_comissao','valor_comissao','taxa_receita','valor_receita']].astype(float)

In [30]:
pd.to_datetime(df_sales[['data_venda','data_primeira_parcela','data_ultima_parcela']])

ValueError: to assemble mappings requires at least that [year, month, day] be specified: [day,month,year] is missing

In [36]:
df_sales[['data_venda','data_primeira_parcela','data_ultima_parcela']].astype('datetime64[ns]')

,data_venda,data_primeira_parcela,data_ultima_parcela
0,2024-06-27,2024-07-10,2024-12-10
1,2024-06-29,2024-07-10,2025-07-10


In [29]:
df_sales

,id_venda,id_prod_caract,nome_adm,nome_produto,id_funcionario,nome_funcionario,grupo,cota,projeto,nome_cliente,data_venda,data_venda_abv,data_primeira_parcela,data_ultima_parcela,valor_venda,taxa_comissao,valor_comissao,taxa_receita,valor_receita,id_income
0,2,10,BR,BR - Serv,1,Andyara,43,122,Especial 1,a,2024-06-27,2024/06,2024-07-10,2024-12-10,30000.0,1.7,510.0,4.0,1200.0,2
1,3,306,Remaza,RMZ Big 75,1,Andyara,44,1,Especial 3,b,2024-06-29,2024/06,2024-07-10,2025-07-10,10000.0,1.5,150.0,4.5,450.0,46


In [28]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id_venda               2 non-null      int64  
 1   id_prod_caract         2 non-null      int64  
 2   nome_adm               2 non-null      object 
 3   nome_produto           2 non-null      object 
 4   id_funcionario         2 non-null      int64  
 5   nome_funcionario       2 non-null      object 
 6   grupo                  2 non-null      int64  
 7   cota                   2 non-null      int64  
 8   projeto                2 non-null      object 
 9   nome_cliente           2 non-null      object 
 10  data_venda             2 non-null      object 
 11  data_venda_abv         2 non-null      object 
 12  data_primeira_parcela  2 non-null      object 
 13  data_ultima_parcela    2 non-null      object 
 14  valor_venda            2 non-null      float64
 15  taxa_comis

In [ ]:
df.apply(lambda x: x.astype(float) if type(x) == )

In [3]:
df = pd.DataFrame({'col1': ['1','2','3'], 'col2': [4,5,6], 'col3': [7,8,9], 'col4': ['a','b','c']})
selected_cols = ['col1'] + list(df.columns[2:])
selected_cols

['col1', 'col3', 'col4']

In [12]:
df[df['col1'] == '2']['col3'].values[0]

8

ValueError: Unable to parse string "a" at position 0

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   col1    3 non-null      object
 1   col2    3 non-null      int64 
 2   col3    3 non-null      int64 
 3   col4    3 non-null      object
dtypes: int64(2), object(2)
memory usage: 224.0+ bytes


In [9]:
for row in df.itertuples():
    for r in row:
        print(f'{r}: {type(r)}')

0: <class 'int'>
1: <class 'str'>
4: <class 'int'>
7: <class 'int'>
a: <class 'str'>
1: <class 'int'>
2: <class 'str'>
5: <class 'int'>
8: <class 'int'>
b: <class 'str'>
2: <class 'int'>
3: <class 'str'>
6: <class 'int'>
9: <class 'int'>
c: <class 'str'>


In [5]:
conn = psycopg2.connect(dbname='postgres', user='postgres', password='1673', host='localhost', port='5432')
cur = conn.cursor()

In [ ]:
# query = f"create table administrators (id_adm int primary key, nome_adm varchar, company_id varchar);"
query = f"insert into administrators (id_adm, nome_adm, company_id) values (1, 'BR', 'abc');"
cur.execute(query)
conn.commit()

In [16]:
company_id = 'abc'

In [36]:
cur.execute(f"SELECT * FROM sales WHERE company_id = '{company_id}';")
sales_list = list(cur.fetchall())
table_sales_columns = [desc[0] for desc in cur.description]
df_sales = pd.DataFrame(sales_list, columns=table_sales_columns)
df_sales.drop(columns=['company_id'], inplace=True)
df_sales['data_venda'] = pd.to_datetime(df_sales['data_venda'], format='%Y-%m-%d')
df_sales['data_venda'] = df_sales['data_venda'].map(lambda x: x.date())
df_sales['valor_venda'] = df_sales['valor_venda'].astype(float)
cur.execute(f"SELECT * FROM prod_caract WHERE company_id = '{company_id}';")
prod_caract_list = list(cur.fetchall())
table_prod_caract_columns = [desc[0] for desc in cur.description]
df_prod_caract = pd.DataFrame(prod_caract_list, columns=table_prod_caract_columns)
df_prod_caract.drop(columns=['company_id'], inplace=True)

In [37]:
df_sales_upd = pd.merge(df_sales, df_prod_caract[['id_prod_caract', 'qtd_parcelas', 'taxa_parcelas']])
df_sales_upd

,id_venda,id_prod_caract,nome_produto,id_funcionario,nome_funcionario,projeto,nome_cliente,data_venda,data_venda_abv,data_primeira_parcela,data_ultima_parcela,valor_venda,taxa_comissao,valor_comissao,valor_liquido,qtd_parcelas,taxa_parcelas
0,1,2,BR - Im/Auto,1,Andy,Network,a,2024-02-01,2024/02,2024-03-10,2024-08-10,50000.0,1.5,750.0,49250.0,6,0.5 - 0.0 - 0.0 - 0.5 - 0.0 - 0.5
1,2,4,BR - Im/Auto,2,Beto Andrade,Especial 2,a,2024-01-09,2024/01,2024-02-10,2024-07-10,50000.0,2.0,1000.0,49000.0,6,0.6 - 0.0 - 0.0 - 0.6 - 0.0 - 0.8
2,3,1,BR - Im/Auto,3,Brenda,Externo,a,2023-12-01,2023/12,2024-01-10,2024-04-10,50000.0,0.7,350.0,49650.0,4,0.4 - 0.0 - 0.0 - 0.3


In [71]:
(dt(2024,2,20).date() - dt(2024,8,10).date()).days/30

-5.733333333333333

In [5]:
cur.execute(f"SELECT s.id_funcionario, s.nome_funcionario, s.nome_produto, s.projeto, s.data_venda, s.data_primeira_parcela, s.data_ultima_parcela, s.valor_venda, s.valor_comissao, \
                pc.qtd_parcelas, pc.taxa_parcelas FROM sales AS s LEFT JOIN prod_caract AS pc on s.id_prod_caract = pc.id_prod_caract;")
prod_sales_list = list(cur.fetchall())
table_prod_sales_columns= [desc[0] for desc in cur.description]
df_prod_sales = pd.DataFrame(prod_sales_list, columns=table_prod_sales_columns)

In [6]:
df_prod_sales

,id_funcionario,nome_funcionario,nome_produto,projeto,data_venda,data_primeira_parcela,data_ultima_parcela,valor_venda,valor_comissao,qtd_parcelas,taxa_parcelas


In [61]:
from datetime import timedelta, datetime as dt
from dateutil.relativedelta import relativedelta

In [8]:
data = dt.now() + relativedelta(months=-2)
data

datetime.datetime(2023, 11, 30, 19, 52, 25, 644139)

In [13]:
dic.values()

dict_values(['d', 4])

In [14]:
df = pd.DataFrame({'col1':[1,2,3], 'col2': ['a','b','c']})
dic = {'col2':'d', 'col1':4}
pd.concat([df, pd.DataFrame([dic])], ignore_index=True)

,col1,col2
0,1,a
1,2,b
2,3,c
3,4,d


In [1]:
lista = []
max(lista)

ValueError: max() arg is an empty sequence

In [25]:
df = pd.DataFrame({'col1': [1,2,3], 'col2': None})
df.iloc[2]['col2'] = 'a'
df


C:\Users\joaopedro\AppData\Local\Temp\ipykernel_6664\3039820842.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[2]['col2'] = 'a'


,col1,col2
0,1,None
1,2,None
2,3,None
